<a href="https://colab.research.google.com/github/VladShajdulin/OTUS/blob/main/home_work_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [59]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from sklearn.model_selection import train_test_split

In [60]:
low, high = -10, 10
n_size = 20000

torch.manual_seed(142)
X = (high - low) * torch.rand((n_size, 2), dtype=torch.float64) + low
y = torch.sin(X[:,0] + 2 * X[:,1]) * torch.exp(-(2 * X[:,0] + X[:,1]) ** 2)
y = torch.unsqueeze(y, -1)

In [61]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=142)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=142)

print(f'Training samples: {X_train.size()[0]}')
print(f'Validating samples: {X_val.size()[0]}')
print(f'Training samples: {X_test.size()[0]}')

Training samples: 14000
Validating samples: 3000
Training samples: 3000


In [62]:
class XY_dataset(Dataset):
  def __init__(self, X, y):
    self.X = X
    self.y = y

  def __len__(self):
    return self.X.size()[0]

  def __getitem__(self, idx):
    return self.X[idx], self.y[idx]

In [77]:
def train(dataloader, model, loss_fn, optimizer, device):
  size = len(dataloader.dataset)
  model.train()
  for batch, (X, y) in enumerate(dataloader):
      X, y = X.to(device), y.to(device)

      # Compute prediction error
      pred = model(X)
      loss = loss_fn(pred, y)

      # Backpropagation
      loss.backward()
      optimizer.step()
      optimizer.zero_grad()

      if batch % 10 == 0:
          loss, current = loss.item(), (batch + 1) * len(X)
          print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def val(X, y, model, loss_fn):
  model.eval()
  with torch.no_grad():
    pred = model(X)
    loss = loss_fn(pred, y).item()
    rel_error = abs(y - pred).mean() / abs(y).mean()

  print(f"\n Val loss: {loss:>7f}, mean relative error: {rel_error:>5f} \n")

In [89]:
model_1 = nn.Sequential(
    nn.Linear(2, 20, dtype=torch.float64),
    nn.Softsign(),
    nn.Linear(20, 20, dtype=torch.float64),
    nn.Softsign(),
    nn.Linear(20, 10, dtype=torch.float64),
    nn.Softsign(),
    nn.Linear(10, 1, dtype=torch.float64)
)

In [92]:
batch_size = 256

train_loader = XY_dataset(X_train, y_train)
train_loader = DataLoader(train_loader, shuffle=True, batch_size=batch_size)
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else 'cpu'
print('Device', device)

Device cpu


In [96]:
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model_1.parameters(), lr=1e-5)
epochs = 10

for t in range(epochs):
  print(f"Epoch {t+1}\n-------------------------------")
  train(train_loader, model_1, loss_fn, optimizer, device)
  val(X_val, y_val, model_1, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 0.016614  [  256/14000]
loss: 0.019762  [ 2816/14000]
loss: 0.018071  [ 5376/14000]
loss: 0.016339  [ 7936/14000]
loss: 0.017288  [10496/14000]
loss: 0.014980  [13056/14000]

 Val loss: 0.015918, mean relative error: 1.233008 

Epoch 2
-------------------------------
loss: 0.015198  [  256/14000]
loss: 0.014535  [ 2816/14000]
loss: 0.011093  [ 5376/14000]
loss: 0.012702  [ 7936/14000]
loss: 0.013052  [10496/14000]
loss: 0.019879  [13056/14000]

 Val loss: 0.015913, mean relative error: 1.231868 

Epoch 3
-------------------------------
loss: 0.008144  [  256/14000]
loss: 0.018846  [ 2816/14000]
loss: 0.016714  [ 5376/14000]
loss: 0.013671  [ 7936/14000]
loss: 0.013744  [10496/14000]
loss: 0.011333  [13056/14000]

 Val loss: 0.015910, mean relative error: 1.229230 

Epoch 4
-------------------------------
loss: 0.026441  [  256/14000]
loss: 0.025604  [ 2816/14000]
loss: 0.016720  [ 5376/14000]
loss: 0.020985  [ 7936/14000]
loss: 0.016649  [1

In [97]:
resid = y_test - model_1(X_test)
resid

tensor([[ 0.0028],
        [-0.0070],
        [ 0.0884],
        ...,
        [ 0.0184],
        [ 0.0039],
        [ 0.0036]], dtype=torch.float64, grad_fn=<SubBackward0>)